# Sparse Kernel Machines

## Abstract

[Link to source code](https://github.com/EpicET/EpicET.github.io/blob/main/posts/blog7/kernel_logistic.py)

In this post, we explore sparse kernel logistic regression using radial basis function (RBF) kernels. We demonstrate how sparsity naturally emerges in these models, meaning only a subset of training points — the support vectors — contribute significantly to the final decision function. We examine how key hyperparameters like the regularization strength (𝜆) and the kernel sharpness (𝛾) influence model behavior, including sparsity, decision boundaries, and overfitting. Through several experiments, we visualize decision surfaces, investigate the impact of parameter choices, and show how kernel methods can capture nonlinear patterns effectively. To evaluate generalization, we conclude with an overfitting case study using ROC curves to compare training and test performance.


In [ ]:
%load_ext autoreload
%autoreload 2

Classification and kernel code adapted from Prof. Phil


In [ ]:
import torch
from matplotlib import pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")


def classification_data(n_points=300, noise=0.2, p_dims=2):

    y = torch.arange(n_points) >= int(n_points / 2)
    y = 1.0 * y
    X = y[:, None] + torch.normal(0.0, noise, size=(n_points, p_dims))
    # X = torch.cat((X, torch.ones((X.shape[0], 1))), 1)

    X = X - X.mean(dim=0, keepdim=True)
    return X, y


def plot_classification_data(X, y, ax):
    assert X.shape[1] == 2, "This function only works for data created with p_dims == 2"
    targets = [0, 1]
    markers = ["o", ","]
    for i in range(2):
        ix = y == targets[i]
        ax.scatter(
            X[ix, 0],
            X[ix, 1],
            s=20,
            c=y[ix],
            facecolors="none",
            edgecolors="darkgrey",
            cmap="BrBG",
            vmin=-1,
            vmax=2,
            alpha=0.8,
            marker=markers[i],
        )
    ax.set(xlabel=r"$x_1$", ylabel=r"$x_2$")


fig, ax = plt.subplots(1, 1)
X, y = classification_data(n_points=100, noise=0.4)
plot_classification_data(X, y, ax)

In [ ]:
def rbf_kernel(X_1, X_2, gamma):
    return torch.exp(-gamma * torch.cdist(X_1, X_2) ** 2)

In [ ]:
from kernel_logistic import KernelLogisticRegression

KR = KernelLogisticRegression(rbf_kernel, lam = 0.1, gamma = 1)
KR.fit(X, y, m_epochs = 500000, lr = 0.0001)

In [ ]:
(1.0 * (KR.a > 0.001)).mean()

In [ ]:
ix = torch.abs(KR.a) > 0.001

x1 = torch.linspace(X[:, 0].min() - 0.2, X[:, 0].max() + 0.2, 101)
x2 = torch.linspace(X[:, 1].min() - 0.2, X[:, 1].max() + 0.2, 101)

X1, X2 = torch.meshgrid(x1, x2, indexing="ij")

x1 = X1.ravel()
x2 = X2.ravel()

X_ = torch.stack((x1, x2), dim=1)

preds = KR.prediction(X_, recompute_kernel=True)
preds = 1.0 * torch.reshape(preds, X1.size())

fig, ax = plt.subplots(1, 1)
ax.contourf(
    X1,
    X2,
    preds,
    origin="lower",
    cmap="BrBG",
    vmin=2 * preds.min() - preds.max(),
    vmax=2 * preds.max() - preds.min(),
)
plot_classification_data(X, y, ax)
plt.scatter(X[ix, 0], X[ix, 1], facecolors="none", edgecolors="black")

## Basic Experiments

### Experiment 1: Large lambda

When 𝜆 is very large, there may be only one point in the training data with a wieght distinguishable from zero.


In [ ]:
KR = KernelLogisticRegression(rbf_kernel, lam=1000, gamma=1)
KR.fit(X, y, m_epochs=500000, lr=0.0001)

In [ ]:
ix = torch.abs(KR.a) > 0.001

x1 = torch.linspace(X[:, 0].min() - 0.2, X[:, 0].max() + 0.2, 101)
x2 = torch.linspace(X[:, 1].min() - 0.2, X[:, 1].max() + 0.2, 101)

X1, X2 = torch.meshgrid(x1, x2, indexing="ij")

x1 = X1.ravel()
x2 = X2.ravel()

X_ = torch.stack((x1, x2), dim=1)

preds = KR.prediction(X_, recompute_kernel=True)
preds = 1.0 * torch.reshape(preds, X1.size())

fig, ax = plt.subplots(1, 1)
ax.contourf(
    X1,
    X2,
    preds,
    origin="lower",
    cmap="BrBG",
    vmin=2 * preds.min() - preds.max(),
    vmax=2 * preds.max() - preds.min(),
)
plot_classification_data(X, y, ax)
plt.scatter(X[ix, 0], X[ix, 1], facecolors="none", edgecolors="black")

### Experiment 2: Changing gamma

Change 𝛄 can result in wigglier descision boundaries.


In [ ]:
for i in range(10):
    KR = KernelLogisticRegression(rbf_kernel, lam=0.1, gamma=i)
    KR.fit(X, y, m_epochs=200000, lr=0.0001)

    ix = torch.abs(KR.a) > 0.001

    x1 = torch.linspace(X[:, 0].min() - 0.2, X[:, 0].max() + 0.2, 101)
    x2 = torch.linspace(X[:, 1].min() - 0.2, X[:, 1].max() + 0.2, 101)

    X1, X2 = torch.meshgrid(x1, x2, indexing="ij")

    x1 = X1.ravel()
    x2 = X2.ravel()

    X_ = torch.stack((x1, x2), dim=1)

    preds = KR.prediction(X_, recompute_kernel=True)
    preds = 1.0 * torch.reshape(preds, X1.size())

    fig, ax = plt.subplots(1, 1)
    ax.contourf(
        X1,
        X2,
        preds,
        origin="lower",
        cmap="BrBG",
        vmin=2 * preds.min() - preds.max(),
        vmax=2 * preds.max() - preds.min(),
    )
    plot_classification_data(X, y, ax)
    plt.scatter(X[ix, 0], X[ix, 1], facecolors="none", edgecolors="black")

### Experiment 3: Dealing with nonlinear data

When the data has a nonlinear pattern (e.g. as generated by sklearn.datasets.make_moons), kernel methods with appropriate parameters can find this pattern effectively.


In [ ]:
from sklearn.datasets import make_moons, make_circles, make_blobs

X, y = make_moons(n_samples=100, noise=0.4)

In [ ]:
KR = KernelLogisticRegression(rbf_kernel, lam=0.1, gamma=1)
KR.fit(X, y, m_epochs=200000, lr=0.0001)

In [ ]:
ix = torch.abs(KR.a) > 0.001

x1 = torch.linspace(X[:, 0].min() - 0.2, X[:, 0].max() + 0.2, 101)
x2 = torch.linspace(X[:, 1].min() - 0.2, X[:, 1].max() + 0.2, 101)

X1, X2 = torch.meshgrid(x1, x2, indexing="ij")

x1 = X1.ravel()
x2 = X2.ravel()

X_ = torch.stack((x1, x2), dim=1)

preds = KR.prediction(X_, recompute_kernel=True)
preds = 1.0 * torch.reshape(preds, X1.size())

fig, ax = plt.subplots(1, 1)
ax.contourf(
    X1,
    X2,
    preds,
    origin="lower",
    cmap="BrBG",
    vmin=2 * preds.min() - preds.max(),
    vmax=2 * preds.max() - preds.min(),
)
plot_classification_data(X, y, ax)
plt.scatter(X[ix, 0], X[ix, 1], facecolors="none", edgecolors="black")

## Overfitting

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# Generate and split data
X, y = classification_data(n_points=300, noise=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Fit a model with low regularization (likely to overfit)
KR = KernelLogisticRegression(rbf_kernel, lam=1e-5, gamma=10)
KR.fit(X_train, y_train, m_epochs=300000, lr=0.0001)

# Predict probabilities on train and test
with torch.no_grad():
    y_train_scores = KR.prediction(X_train)
    y_test_scores = KR.prediction(X_test)

# Convert to NumPy for sklearn
y_train_np = y_train.numpy()
y_test_np = y_test.numpy()
train_scores_np = y_train_scores.numpy()
test_scores_np = y_test_scores.numpy()

# Compute ROC curves
fpr_train, tpr_train, _ = roc_curve(y_train_np, train_scores_np)
fpr_test, tpr_test, _ = roc_curve(y_test_np, test_scores_np)

# Compute AUCs
auc_train = auc(fpr_train, tpr_train)
auc_test = auc(fpr_test, tpr_test)

# Plot
plt.figure(figsize=(7, 5))
plt.plot(fpr_train, tpr_train, label=f"Train ROC (AUC = {auc_train:.2f})")
plt.plot(fpr_test, tpr_test, label=f"Test ROC (AUC = {auc_test:.2f})", linestyle='--')
plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves: Overfitting Example")
plt.legend()
plt.grid(True)
plt.show()

In this experiment, we simulate the problem of overfitting by training a kernel logistic regression model with extremely low regularization (very small lambda). This allows the model to fit the training data very closely, but it generalizes poorly to unseen test data.

We evaluate performance using ROC curves and the Area Under the Curve (AUC) metric. A large gap between train and test AUCs is a strong signal of overfitting.


## Discussion

- Summary of blog post

